In [ ]:
from huggingface_hub import login

login(
  token="", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM, PeftModel, PeftConfig
#from trl import SFTTrainer
import bitsandbytes as bnb
from transformers import pipeline

In [ ]:
# Seed the same seed to all
import random
import numpy as np
import gc
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

SEED = 42
seed_everything(SEED)

In [ ]:
torch_dtype=torch.float16

In [ ]:
def load_model(model_name, attn_implementation="sdpa"):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        #quantization_config=bnb_config,
        torch_dtype=torch.float16,
        device_map="auto", # dispatch efficiently the model on the available ressources
        max_memory = {i: max_memory for i in range(n_gpus)},
        attn_implementation = attn_implementation
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

    return model, tokenizer

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"

model, tokenizer = load_model(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:814: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "system", "content": """
You are a helpful chatbot that answers user queries """},
    {"role": "user", "content": "What do you know about NVIDIA’s Jetson Orin"},
]

In [ ]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_special_tokens=True)

In [ ]:
prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a helpful chatbot that answers user queries<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat do you know about NVIDIA’s Jetson Orin<|eot_id|>'

In [ ]:
# print(tokenizer.decode(prompt))

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")

In [ ]:
inputs

{'input_ids': tensor([[128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,
           2696,     25,   6790,    220,   2366,     18,    198,  15724,   2696,
             25,    220,   1627,  10263,    220,   2366,     19,    271,   2675,
            527,    264,  11190,   6369,   6465,    430,  11503,   1217,  20126,
         128009, 128006,    882, 128007,    271,   3923,    656,    499,   1440,
            922,  34661,    753,  25365,    942,   2582,    258, 128009]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])}

In [ ]:
with torch.no_grad(): ## Setting our model in inference mode
  generation_output = model.generate(
      input_ids=inputs.input_ids.to("cuda"),
      return_dict_in_generate=True,
      output_scores=True,
      #pad_token_id = tokenizer.pad_token_id
      max_new_tokens=300,
  )
print('\nAnswer: ', tokenizer.decode(generation_output.sequences[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Answer:  <|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful chatbot that answers user queries<|eot_id|><|start_header_id|>user<|end_header_id|>

What do you know about NVIDIA’s Jetson Orin<|eot_id|><|start_header_id|>assistant<|end_header_id|>

NVIDIA's Jetson Orin is a high-performance, AI computing module designed for edge AI applications. It's the fourth generation of NVIDIA's Jetson platform, succeeding the Jetson Xavier series. Here are some key features and specifications of the Jetson Orin:

**Key Features:**

1.  **AI Performance**: The Jetson Orin is powered by an NVIDIA Ampere GPU architecture, which provides up to 5x the AI performance of the previous generation.
2.  **Multi-Instance Support**: It supports up to 4 instances of the same model, enabling the deployment of multiple AI models on a single module.
3.  **Dual-Channel Memory**: The module features a dual-chann